In [36]:
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time

from scipy.sparse import coo_matrix
from dgl.nn import GraphConv

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [45]:
class GCN(nn.Module):
    def __init__(self, l1_in_feats, l1_out_feats, l2_in_feats, l2_out_feats):
        super(GCN, self).__init__()
        self.layer1 = GraphConv(l1_in_feats, l1_out_feats)
        self.layer2 = GraphConv(l2_in_feats, l2_out_feats)
    
    def forward(self, g, in_feats):
        h = F.relu(self.layer1(g, in_feats))
        h = self.layer2(g, h)
        return h
        

In [46]:
def load_dataset(d,device):
    g = d[0].to(device)
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask

# Experiments - Measure Inference Time

# Cora

In [51]:
model = GCN(1433,16,16,7)
model.load_state_dict(torch.load('data/weight/cora.pt'))
model.eval().to(device)


GCN(
  (layer1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [52]:
dataset = dgl.data.CoraGraphDataset() # import dataset
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
g.add_edges(g.nodes(), g.nodes())

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [53]:
start,end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timer = np.zeros((repetitions, 1))

In [60]:
# WARM UP
for _ in range(10):
    _ = model(g,feats)
    
# EXPERIMENT STARTS (milliseconds)
for rep in range(repetitions):
    start.record()
    _ = model(g,feats)
    end.record()
    
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timer[rep] = curr_time

print(timer)

[[2.70579195]
 [2.66729593]
 [2.67811203]
 [2.67462397]
 [2.67248011]
 [2.67286396]
 [2.6652801 ]
 [2.67398405]
 [2.66755199]
 [2.66969609]
 [2.67388797]
 [2.66188788]
 [2.6580801 ]
 [2.67007995]
 [2.66886401]
 [2.65891194]
 [2.67286396]
 [2.66687989]
 [2.66041589]
 [2.66406393]
 [2.66511989]
 [2.6639359 ]
 [2.65603209]
 [2.66294408]
 [2.66444802]
 [2.66336012]
 [2.7105279 ]
 [2.66655993]
 [2.65798402]
 [2.65158391]
 [2.66473603]
 [2.6599679 ]
 [2.65977597]
 [2.65670395]
 [2.6624639 ]
 [2.6527679 ]
 [2.65724802]
 [2.66582394]
 [2.65644789]
 [2.66358399]
 [2.67337608]
 [2.6555841 ]
 [2.65516806]
 [2.66281605]
 [2.66390395]
 [2.65001607]
 [2.66217589]
 [2.67052794]
 [2.658144  ]
 [2.66419196]
 [2.66438389]
 [2.66400003]
 [2.66620803]
 [2.66073608]
 [2.66707206]
 [2.65462399]
 [2.65587211]
 [2.66291189]
 [2.64860797]
 [2.66236806]
 [2.66316795]
 [2.66224003]
 [2.65516806]
 [2.70121598]
 [2.67139196]
 [2.72607994]
 [2.65993595]
 [2.66611195]
 [2.66163206]
 [2.65670395]
 [2.66780806]
 [2.66

# Citeseer

In [61]:
model = GCN(3703,16,16,6)
model.load_state_dict(torch.load('data/weight/citeseer.pt'))
model.eval().to(device)


GCN(
  (layer1): GraphConv(in=3703, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=6, normalization=both, activation=None)
)

In [62]:
dataset = dgl.data.CiteseerGraphDataset() # import dataset
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
g.add_edges(g.nodes(), g.nodes())

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [63]:
start,end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timer = np.zeros((repetitions, 1))

In [64]:
# WARM UP
for _ in range(10):
    _ = model(g,feats)
    
# EXPERIMENT STARTS (milliseconds)
for rep in range(repetitions):
    start.record()
    _ = model(g,feats)
    end.record()
    
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timer[rep] = curr_time

print(timer)

[[3.05020809]
 [2.7700479 ]
 [2.82175994]
 [2.77814388]
 [2.80140805]
 [2.78505611]
 [2.78031993]
 [2.76678395]
 [2.78134394]
 [2.77753592]
 [2.7794559 ]
 [2.77519989]
 [2.76579189]
 [2.76678395]
 [2.79190397]
 [2.77343988]
 [2.77744007]
 [2.77334404]
 [2.76150393]
 [2.76979208]
 [2.77887988]
 [2.76959991]
 [2.76985598]
 [2.75948811]
 [2.76719999]
 [2.81020808]
 [2.77702403]
 [2.76537609]
 [2.76089597]
 [2.76304007]
 [2.777632  ]
 [2.7722559 ]
 [2.7768321 ]
 [2.76297593]
 [2.766752  ]
 [2.77305603]
 [2.77315211]
 [2.76783991]
 [2.77708793]
 [2.75999999]
 [2.77033591]
 [2.76915193]
 [2.77635193]
 [2.77196789]
 [2.76995206]
 [2.77168012]
 [5.39395189]
 [8.84876823]
 [2.7837441 ]
 [2.78572798]
 [2.81001592]
 [2.77616   ]
 [2.77283192]
 [2.76259208]
 [2.77392006]
 [2.77478409]
 [2.78067207]
 [2.77471995]
 [2.8360641 ]
 [2.83516788]
 [2.77142406]
 [2.78518391]
 [2.77968001]
 [2.77964807]
 [2.78009605]
 [2.77209592]
 [2.770432  ]
 [2.77030396]
 [2.77635193]
 [2.77420807]
 [2.76195192]
 [2.75

# Pubmed

In [66]:
model = GCN(500,16,16,3)
model.load_state_dict(torch.load('data/weight/pubmed.pt'))
model.eval().to(device)


dataset = dgl.data.PubmedGraphDataset() # import dataset
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
g.add_edges(g.nodes(), g.nodes())

start,end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timer = np.zeros((repetitions, 1))

# WARM UP
for _ in range(10):
    _ = model(g,feats)
    
# EXPERIMENT STARTS (milliseconds)
for rep in range(repetitions):
    start.record()
    _ = model(g,feats)
    end.record()
    
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timer[rep] = curr_time

print(timer)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
[[2.72796798]
 [2.69491196]
 [2.69987202]
 [2.69270396]
 [2.69491196]
 [2.68441606]
 [2.68521595]
 [2.6967361 ]
 [2.68294406]
 [2.68931198]
 [2.69155192]
 [2.68940806]
 [2.6707201 ]
 [2.69039989]
 [2.72883201]
 [2.68736005]
 [2.68156791]
 [2.68768001]
 [2.68531203]
 [2.68358397]
 [2.69507194]
 [2.68796802]
 [2.68204808]
 [2.67984009]
 [2.68726397]
 [2.68255997]
 [2.67062402]
 [2.68556809]
 [2.68032002]
 [2.68390393]
 [2.67561603]
 [2.68313599]
 [2.6818881 ]
 [2.68019199]
 [2.68006396]
 [2.68284798]
 [2.67840004]
 [2.67462397]
 [2.67977595]
 [2.67683196]
 [2.68457603]
 [2.67849588]
 [3.27590394]
 [2.69120002]
 [2.69158411]
 [2.6840961 ]
 [2.67705607]
 [2.67775989]
 [2.68607998]
 [2.69382405]
 [2.67859197]
 [2.69023991]
 [2.67529607]
 [2.69382405]
 [2.68176007]
 [2.684448  ]
 [2.68291211]
 [2.68095994]
 [2.684

# Reddit

In [67]:
model = GCN(602,16,16,41)
model.load_state_dict(torch.load('data/weight/reddit.pt'))
model.eval().to(device)


dataset = dgl.data.RedditDataset() # import dataset
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
g.add_edges(g.nodes(), g.nodes())

start,end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 300
timer = np.zeros((repetitions, 1))

# WARM UP
for _ in range(10):
    _ = model(g,feats)
    
# EXPERIMENT STARTS (milliseconds)
for rep in range(repetitions):
    start.record()
    _ = model(g,feats)
    end.record()
    
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timer[rep] = curr_time

print(timer)

[[73.32259369]
 [73.19213104]
 [72.59008026]
 [73.57289886]
 [73.06345367]
 [73.24380493]
 [73.38658905]
 [72.67424011]
 [72.9825592 ]
 [73.46092987]
 [73.41254425]
 [73.25910187]
 [73.21491241]
 [73.12806702]
 [73.26713562]
 [73.0076828 ]
 [72.50540924]
 [73.02841949]
 [73.25769806]
 [72.99552155]
 [73.2337265 ]
 [73.11577606]
 [73.36029053]
 [73.50624084]
 [72.99676514]
 [72.96089935]
 [73.15449524]
 [73.17155457]
 [72.88963318]
 [73.11017609]
 [73.89849854]
 [73.3432312 ]
 [72.73046112]
 [73.19631958]
 [72.75411224]
 [73.08140564]
 [73.5254364 ]
 [73.14457703]
 [73.32102203]
 [73.29058838]
 [72.91149139]
 [73.00201416]
 [73.08326721]
 [73.28141022]
 [72.82214355]
 [72.96876526]
 [73.50415802]
 [72.98249817]
 [72.53068542]
 [72.89910126]
 [73.20505524]
 [73.35443115]
 [73.14479828]
 [73.15939331]
 [73.09670258]
 [73.12928009]
 [73.139328  ]
 [72.99008179]
 [72.91702271]
 [73.04799652]
 [73.19881439]
 [73.13378906]
 [73.01475525]
 [73.22358704]
 [72.9187851 ]
 [72.95574188]
 [73.77257

# Experiment (Multi-GPU Inference Time)

In [ ]:
def init_process_group(world_size, rank):
    dist.init_process_group(
        backend='nccl'
        init_method='tcp://127.0.0.1:12345',
        world_size=world_size,
        rank=rank)